In [1]:
#env verification

import sys
sys.executable

'd:\\Sentio\\venv\\Scripts\\python.exe'

In [ ]:
#loading dataset

import pandas as pd
df = pd.read_csv("../data/emotional_dataset.csv")
df.head()

,Unnamed: 0,Situation,emotion,empathetic_dialogues,labels,Unnamed: 5,Unnamed: 6
0,0,I remember going to the fireworks with my best...,sentimental,Customer :I remember going to see the firework...,"Was this a friend you were in love with, or ju...",NaN,NaN
1,1,I remember going to the fireworks with my best...,sentimental,Customer :This was a best friend. I miss her.\...,Where has she gone?,NaN,NaN
2,2,I remember going to the fireworks with my best...,sentimental,Customer :We no longer talk.\nAgent :,Oh was this something that happened because of...,NaN,NaN
3,3,I remember going to the fireworks with my best...,sentimental,Customer :Was this a friend you were in love w...,This was a best friend. I miss her.,NaN,NaN
4,4,I remember going to the fireworks with my best...,sentimental,Customer :Where has she gone?\nAgent :,We no longer talk.,NaN,NaN


In [13]:
#checking for null values

df.shape
df.columns

Index(['Situation', 'emotion', 'empathetic_dialogues', 'labels', 'Unnamed: 5',
       'Unnamed: 6'],
      dtype='str')

In [ ]:
#droping useless columns

df = df.drop(columns=["Unnamed: 5", "Unnamed: 6"])
df.head()

,Situation,emotion,empathetic_dialogues,labels
0,I remember going to the fireworks with my best...,sentimental,Customer :I remember going to see the firework...,"Was this a friend you were in love with, or ju..."
1,I remember going to the fireworks with my best...,sentimental,Customer :This was a best friend. I miss her.\...,Where has she gone?
2,I remember going to the fireworks with my best...,sentimental,Customer :We no longer talk.\nAgent :,Oh was this something that happened because of...
3,I remember going to the fireworks with my best...,sentimental,Customer :Was this a friend you were in love w...,This was a best friend. I miss her.
4,I remember going to the fireworks with my best...,sentimental,Customer :Where has she gone?\nAgent :,We no longer talk.


In [18]:
#renaming columns

df = df.rename(columns={
    "Situation": "situation",
    "emotion": "emotion",
    "empathetic_dialogues": "dialogue",
    "labels": "response"
})

df.columns

Index(['situation', 'emotion', 'dialogue', 'response'], dtype='str')

In [19]:
#clean dialogue text

df["dialogue"] = (
    df["dialogue"]
    .str.replace("Customer:", "", regex=False)
    .str.replace("Agent:", "", regex=False)
    .str.replace("\n", " ", regex=False)
    .str.strip()
)

df["dialogue"].head()

0    Customer :I remember going to see the firework...
1    Customer :This was a best friend. I miss her. ...
2                 Customer :We no longer talk. Agent :
3    Customer :Was this a friend you were in love w...
4                Customer :Where has she gone? Agent :
Name: dialogue, dtype: str

In [21]:
#checking for null values
df.isnull().sum()

situation    0
emotion      4
dialogue     0
response     0
dtype: int64

In [23]:
#removing null rows

df = df.dropna()
df.shape

(64632, 4)

In [24]:
#9-core emotion taxonomy

emotion_map = {
    # sadness
    "sad": "sadness",
    "lonely": "sadness",
    "disappointed": "sadness",
    "guilty": "sadness",
    "nostalgic": "sadness",

    # anger
    "angry": "anger",
    "annoyed": "anger",
    "furious": "anger",
    "jealous": "anger",
    "disgusted": "anger",

    # fear
    "afraid": "fear",
    "terrified": "fear",
    "anxious": "fear",

    # joy
    "joyful": "joy",
    "excited": "joy",
    "content": "joy",
    "impressed": "joy",

    # love / connection
    "grateful": "love",
    "hopeful": "love",
    "sentimental": "love",

    # anticipation
    "anticipating": "anticipation",
    "prepared": "anticipation",

    # surprise
    "surprised": "surprise",

    # confidence
    "confident": "confidence",
    "proud": "confidence"
}


In [25]:
#appling normalization

df["emotion_normalized"] = df["emotion"].map(emotion_map)
df = df.dropna(subset=["emotion_normalized"])

df["emotion_normalized"].value_counts()

emotion_normalized
anger           10553
sadness         10337
joy              8325
fear             6205
love             5883
confidence       4284
anticipation     3963
surprise         3295
Name: count, dtype: int64

In [27]:
#creating RAG text

df["rag_text"] = (
    "Situation: " + df["situation"] +
    " | Emotion: " + df["emotion_normalized"] +
    " | Dialogue: " + df["dialogue"]
)

df[["rag_text", "response"]].head()


,rag_text,response
0,Situation: I remember going to the fireworks w...,"Was this a friend you were in love with, or ju..."
1,Situation: I remember going to the fireworks w...,Where has she gone?
2,Situation: I remember going to the fireworks w...,Oh was this something that happened because of...
3,Situation: I remember going to the fireworks w...,This was a best friend. I miss her.
4,Situation: I remember going to the fireworks w...,We no longer talk.
